#  BKLM Data Taking - Short Guide

you can find the original Page over [here](https://confluence.desy.de/display/BI/BKLM+Data+Taking+-+Short+Guide).


![VNC Setup](img/kek_vnc.png)


For the time beeing the Controlling of the HV is ignored. 

The setup uses the following Computers/shells.

|Shell |      Computer  |  Use |
|----------|-----------|------|
|KLM01_A   | klm01       | Start/Kill run |
|KLM01_B   | klm01       |   Auxiliary terminal to debug COPPERs |
|ttd11_A   | ttd11       |    Program FEEs; Include / mask sectors; Start reset triggers  |
|IDLAB_A   | idlab       | configure BF2   |
|ttd11_B   | ttd11       | Local run status |
|ttd11_C   | ttd11       | Global run status| 
|KLM01_C   | klm01       | Check COPPERS status|
|KLM01_D   | klm01       | Launch SLC     | 




In [ ]:
import  config.config_hawaii as  config
from Linux_Helpers.remoteshell2 import remoteShell2
from equipment.ScintilatorReadout import ScintilatorReadout

## Setting up the envirement

Since we want to use remote terminals, we have to set them up. Since this notebook is not (yet) connected to the KEK setup all the shells are just dummies and most likley they will produce some errors.

In [ ]:
KLM01_A = remoteShell2(config.klm01)
KLM01_B = remoteShell2(config.klm01)
ttd11_A = remoteShell2(config.ttd11)
IDLAB_A = remoteShell2(config.idlab)
ttd11_B = remoteShell2(config.ttd11)
ttd11_C = remoteShell2(config.ttd11)
KLM01_C = remoteShell2(config.klm01)
KLM01_D = remoteShell2(config.klm01)

Once the Notebook is connected to KEK, 
all of these commands should run without error.
If the command `remoteShell2` produces and error make sure the SSH_Tunnel is setup correctly and to PocketDAQ / Copper is running

# Preliminary steps


## Program the FEEs

- Where? `ttd11_A` (bottom-left terminal, left tab)
- How? launch the following commands:
```
./progfee.sh
./progfee_BB.sh
```


In [ ]:
print(ttd11_A.sendAndRecieve("./progfee.sh"))
print(ttd11_A.sendAndRecieve("./progfee_BB.sh"))

- Check: the last messages shown in the terminal are:

   `ALL KLM FEE PROG DONE!` 
   
   and 
   
   `ALL KLM BB FEE PROG DONE!` 
   
   In the meanwhile, the bottom-center terminal, left tab (`ttd11_B`) should be in an ERROR  state and become purple: it's fine.
   
In a future modification to this Notebook the status of `ttd11_B` should actually be checked.


## Program the BF2 sector

(Note added on 2018-05-01: BF2 is currently masked, so the commands this section can be skipped- Now BB2 is acting as BF2 for debug interface, but script names are the same for ease of use)


- Where? `IDLAB_A` (bottom-left terminal, right tab)
- How? launch the following commands:

```
  impact -batch imp_prog_BF2scrods.cmd
  impact -batch imp_prog_BF2dc.cmd
```




In [ ]:
print(IDLAB_A.sendAndRecieve("impact -batch imp_prog_BF2scrods.cmd"))
print(IDLAB_A.sendAndRecieve("impact -batch imp_prog_BF2dc.cmd"))

- Check: one of the last message shown in the terminal is:

```
INFO:iMPACT - '1': Checking done pin....done.   / /   '1': Programmed successfully.   / /   Elapsed time =  X sec.
```


  Ignore further error messages!  


## Prepare ttd11
- Where? `ttd11_A` (bottom-left terminal, left tab)
- How? launch the following commands:
```        
  resetft -191
```
  With EKLM: 
```  
cd withEKLM; ./prepft_butBF2_withEKLM.sh
```

   Without EKLM: 
```
./prepft_butBF2.sh
```



In [ ]:
print(ttd11_A.sendAndRecieve("resetft -191"))


useEKLM = True
if useEKLM:
    print(ttd11_A.sendAndRecieve("cd withEKLM; ./prepft_butBF2_withEKLM.sh"))
else:
    print(ttd11_A.sendAndRecieve("./prepft_butBF2.sh"))


- Check: 
  
  the last message shown in the terminal is:
```
resetting trigger
```

## Clear "b2linkdown" after reprograming

Usually you will see `b2linkdown` after reprograming, `b2linkdown` shown in the the bottom-center terminal. Run the following:


- At `KLM01_B` run:
```
~/run/scripts/batchTesths  
```

  `batchTesths` does "tesths -abcd" for all four coppers)


- At `ttd11_A` run:
```
./prepft_butBF2.sh
```

In [ ]:
print(KLM01_B.sendAndRecieve("~/run/scripts/batchTesths"))

In [ ]:
print(ttd11_A.sendAndRecieve("./prepft_butBF2.sh"))

then you should able to see `b2linkdown` was cleared, and `FTSW` in yellow READY state.

# Launch the Slow Control

- Where? `KLM01_D` (bottom-right terminal, right tab)
- How? launch the following commands:


```
    ~/run/scripts/load_brcl.sh rcl_rev2_40_th-100  
```

    
    (NOTE: Every time you send this command, you need to resend the following commands)

    NOTE: kill all that `watch` commands which are reading status of registers to avoid any conflict when you issue the load_brcl.sh. Kill the 
  
  ```
    watch ./checkpcr 
```
  
    and 
  
  ```
    watch ./dcregs.sh
```

    Don't keep `watch ./dcregs.sh` running! Unless after reprograming you want to check the status, you run

```
    watch ./dcreg.sh
```

    then kill it right away otherwise you will forget later.  

In [ ]:
print(KLM01_D.sendAndRecieve("~/run/scripts/load_brcl.sh rcl_rev2_40_th-100"))

- Check: the last message shown in the terminal is:

`window fine stop   / /   reg0033 = XXXXXXXX`


- Then run this command to send MPPC gain settings to Layer 1 on Barrel - wait for all windows to finish completely before launching next one
```
cd ~/run/scripts
./load_klm_calset_L1.sh calset_171118_031700 200
```

In [ ]:
print(KLM01_D.sendAndRecieve("cd ~/run/scripts; ./load_klm_calset_L1.sh calset_171118_031700 200"))

- Wait for command to finish (it opens 16 concurrent terminals, so need to have X tunneling)

- after this run the following command:

    `./load_klm_calset_L2.sh calset_171118_060700 200`

In [ ]:
print(KLM01_D.sendAndRecieve("cd ~/run/scripts; ./load_klm_calset_L2.sh calset_171118_060700 200"))

- wait for previous command to completely finish, then set lookback and width for scintilaltor 

- make sure you do not use any other scint. lookback commands after this:
- `./set_addl.sh` (this command is very quiet) (if you see "Device or resource busy" message, just run this command again.)


In [ ]:
print(KLM01_D.sendAndRecieve("cd ~/run/scripts; ./set_addl.sh"))

## Reset the trigger
- Where? `ttd11_A` (bottom-left terminal, left tab)
- How? launch the following command:

```
resetft -191
```


In [ ]:
print(ttd11_A.sendAndRecieve("resetft -191"))

- Check: the bottom-center terminal, left tab `ttd11_B` should pass in a READY state and become yellow 

# Finished "Preliminary steps"
With this the Setup is Ready for Data taking